<a href="https://colab.research.google.com/github/Baekhyeonjeong/movie-review-/blob/main/%EC%98%81%ED%99%94_%EC%9C%84%ED%94%8C%EB%9E%98%EC%89%AC_%EB%A6%AC%EB%B7%B0_%ED%98%95%ED%83%9C%EC%86%8C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [218]:
# konlpy 설치하기
!pip install konlpy

In [219]:
# konlpy를 실행하기
import konlpy

In [220]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [221]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [222]:
## 문서 ID로 실제 파일 불러오기
# 위플래쉬 (1).csv
## https://drive.google.com/file/d/1WOH9ObQl3kJ47zWAWIjXVYjcZpqO2Ldt/view?usp=sharing


rawdata_downloaded = drive.CreateFile({'id': '1WOH9ObQl3kJ47zWAWIjXVYjcZpqO2Ldt'})
rawdata_downloaded.GetContentFile('위플래쉬 (1).csv')

In [223]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('위플래쉬 (1).csv', sep=",")

In [224]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,전설의한창도(manh****),드럼은 쟤가 쳤는데 왜 내가 지칠까?,9,4323,162,2015-03-13 15:45:00,0
1,1,지나갑니다1(m9so****),교수가 좋은 사람인지 나쁜사람인지 헷갈리는 영화,9,3426,215,2015-03-13 23:57:00,0
2,2,stud****,깨져서 나오거나 깨고 나오거나,9,3038,223,2015-03-12 23:04:00,관람객
3,3,altm****,마지막 십분 영환관에선 침 넘기는 소리조차 들릴 정도로 몰입도가 어마어마했음,10,3092,330,2015-03-12 12:21:00,관람객
4,4,보코리(char****),"업스윙윙이 나왔을 때 한방, 카라반 나왔을 때 또 한방....",10,2683,190,2015-03-12 13:26:00,관람객
...,...,...,...,...,...,...,...,...
9075,9075,강한마음(ends****),재즈음악 드럼 아는 분들이나 봐야 재미있음,1,7,21,2015-03-17 00:56:00,0
9076,9076,Chera(opri****),미국판 클레멘타인..,1,10,24,2015-03-15 11:38:00,0
9077,9077,horn****,일점도아까운영화>_<;;;,1,10,24,2015-03-13 08:59:00,0
9078,9078,비이(zzun****),예술과 사랑은 같이 갈 수 없는건가요...?연기는 소름돋도록 잘하는데 솔직히 재미는...,5,4,19,2015-03-13 00:59:00,0


In [225]:
원본데이터NAN삭제 = 원본데이터.dropna(axis=0)
## 바로: NAN값(빈값)을 삭제하기 위한 프로세스

In [226]:
원본데이터NAN삭제

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,전설의한창도(manh****),드럼은 쟤가 쳤는데 왜 내가 지칠까?,9,4323,162,2015-03-13 15:45:00,0
1,1,지나갑니다1(m9so****),교수가 좋은 사람인지 나쁜사람인지 헷갈리는 영화,9,3426,215,2015-03-13 23:57:00,0
2,2,stud****,깨져서 나오거나 깨고 나오거나,9,3038,223,2015-03-12 23:04:00,관람객
3,3,altm****,마지막 십분 영환관에선 침 넘기는 소리조차 들릴 정도로 몰입도가 어마어마했음,10,3092,330,2015-03-12 12:21:00,관람객
4,4,보코리(char****),"업스윙윙이 나왔을 때 한방, 카라반 나왔을 때 또 한방....",10,2683,190,2015-03-12 13:26:00,관람객
...,...,...,...,...,...,...,...,...
9075,9075,강한마음(ends****),재즈음악 드럼 아는 분들이나 봐야 재미있음,1,7,21,2015-03-17 00:56:00,0
9076,9076,Chera(opri****),미국판 클레멘타인..,1,10,24,2015-03-15 11:38:00,0
9077,9077,horn****,일점도아까운영화>_<;;;,1,10,24,2015-03-13 08:59:00,0
9078,9078,비이(zzun****),예술과 사랑은 같이 갈 수 없는건가요...?연기는 소름돋도록 잘하는데 솔직히 재미는...,5,4,19,2015-03-13 00:59:00,0


# 형태소 분석

In [227]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [228]:
# "원본데이터NAN삭제"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터NAN삭제.iterrows(): # 원본데이터NAN삭제의 컬럼(행)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 대상 텍스트의 열은 2번입니다. python은 0부터 숫자를 세게 됩니다!!!
    분류 = row[3] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 평점 정보는 3번째 열에 있습니다.
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [229]:
# "형태소" 변수의 저장 내용 확인
형태소

[('이런', 'Modifier'),
 ('영화', 'Noun'),
 ('뻔하잖냐', 'Adjective'),
 ('90', 'Number'),
 ('2000년', 'Number'),
 ('대도', 'Foreign'),
 ('이런', 'Modifier'),
 ('패턴', 'Noun'),
 ('의', 'Josa'),
 ('음악', 'Noun'),
 ('영화', 'Noun'),
 ('주', 'Modifier'),
 ('구', 'Modifier'),
 ('장창', 'Noun'),
 ('나옴', 'Verb'),
 ('똑같은', 'Adjective'),
 ('레퍼토리', 'Noun')]

In [230]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,이런,Modifier,1,1
1,영화,Noun,1,1
2,뻔하잖냐,Adjective,1,1
3,90,Number,1,1
4,2000년,Number,1,1
5,대도,Foreign,1,1
6,이런,Modifier,1,1
7,패턴,Noun,1,1
8,의,Josa,1,1
9,음악,Noun,1,1


In [231]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[     0            1  type  count
 0   드럼         Noun     9      1
 1    은         Josa     9      1
 2    쟤         Noun     9      1
 3    가         Josa     9      1
 4  쳤는데         Verb     9      1
 5    왜         Noun     9      1
 6    내         Noun     9      1
 7    가         Josa     9      1
 8  지칠까         Verb     9      1
 9    ?  Punctuation     9      1,       0          1  type  count
 0    교수       Noun     9      1
 1     가       Josa     9      1
 2    좋은  Adjective     9      1
 3    사람       Noun     9      1
 4    인지       Josa     9      1
 5  나쁜사람       Noun     9      1
 6    인지       Josa     9      1
 7  헷갈리는  Adjective     9      1
 8    영화       Noun     9      1,       0     1  type  count
 0   깨져서  Verb     9      1
 1  나오거나  Verb     9      1
 2    깨고  Verb     9      1
 3  나오거나  Verb     9      1,          0          1  type  count
 0      마지막       Noun    10      1
 1       십분       Noun    10      1
 2        영   Modifier    10      1
 3       환관 

In [232]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [233]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,드럼,Noun,9,1
1,은,Josa,9,1
2,쟤,Noun,9,1
3,가,Josa,9,1
4,쳤는데,Verb,9,1
...,...,...,...,...
12,구,Modifier,1,1
13,장창,Noun,1,1
14,나옴,Verb,1,1
15,똑같은,Adjective,1,1


In [234]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [235]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,드럼,Noun,9,1
1,은,Josa,9,1
2,쟤,Noun,9,1
3,가,Josa,9,1
4,쳤는데,Verb,9,1
...,...,...,...,...
12,구,Modifier,1,1
13,장창,Noun,1,1
14,나옴,Verb,1,1
15,똑같은,Adjective,1,1


In [236]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [237]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소   품사           분류
     Foreign      10    1
     Foreign      3     1
!     Punctuation  1     5
                   3     6
                   4     1
                        ..
힘주다가  Verb         10    1
힘줘가며  Verb         9     1
힙니    Noun         1     1
힙스터   Noun         1     1
不狂不及  Foreign      10    1
Name: 카운트, Length: 29735, dtype: int64

In [238]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [239]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소  품사          분류     
    Foreign     10    1
    Foreign     3     1
!    Punctuation 1     5
                 3     6
                 4     1
...                  ...
힘주다가 Verb        10    1
힘줘가며 Verb        9     1
힙니   Noun        1     1
힙스터  Noun        1     1
不狂不及 Foreign     10    1

[29735 rows x 1 columns]

In [240]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')